In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import numpy as np

file_path = './test_files/3dbloodnewcolors.txt'
df = pd.read_csv(file_path, header=None)
df.columns = ['x', 'y', 'z', 'r', 'g', 'b']
df

,x,y,z,r,g,b
0,15.897575,-9.944930,9.436969,255,0,255
1,15.126414,-10.350229,9.924628,255,0,255
2,14.322758,-10.811392,10.299265,255,0,255
3,13.450273,-11.210674,10.576853,255,0,255
4,12.503754,-11.472693,10.757642,255,0,255
...,...,...,...,...,...,...
114,5.546314,-8.355754,6.195095,200,0,0
115,4.608029,-8.621447,6.373613,200,0,0
116,3.948629,-9.093957,6.939815,200,0,0
117,3.512919,-9.556698,7.710643,200,0,0


In [24]:
def distance_calculator(df):
    # Drop NaN values (usually the first value)
    dx = df['x'].diff()
    dy = df['y'].diff()
    dz = df['z'].diff()

    # Calculate the Euclidean distance between consecutive rows
    distances = np.sqrt(dx**2 + dy**2 + dz**2)
    df['distance_from_last'] = distances
    return df

df = distance_calculator(df)
# Assign each line a unique id based on the RGB values
df['line_id'] = pd.factorize(df[['r','g','b']].apply(tuple, axis=1))[0]

shape = df[df['line_id'] == 0]

if shape.iloc[-1]['distance_from_last'] > 0.1:
    # Move the last row to the top
    last_row = shape.iloc[[-1]]  # Select the last row as a DataFrame
    remaining_rows = shape.iloc[:-1]  # Select all rows except the last
    shape = pd.concat([last_row, remaining_rows]).reset_index(drop=True)

shape = distance_calculator(shape)

df[df['line_id'] == 0] = shape
df[0:25]

# Challenge is actually that you need to identify common start and end points.





,x,y,z,r,g,b,distance_from_last,line_id
0,2.905744,-10.005951,9.741123,255,0,255,NaN,0
1,2.160242,-9.911603,9.093912,255,0,255,0.991744,0
2,16.350606,-9.862319,9.093912,255,0,255,14.190450,0
3,15.897575,-9.944930,9.436969,255,0,255,0.574238,0
4,15.126414,-10.350229,9.924628,255,0,255,0.998383,0
5,14.322758,-10.811392,10.299265,255,0,255,0.999443,0
6,13.450273,-11.210674,10.576853,255,0,255,0.998856,0
7,12.503754,-11.472693,10.757642,255,0,255,0.998617,0
8,11.519619,-11.609816,10.864519,255,0,255,0.999374,0
9,10.522752,-11.655140,10.924067,255,0,255,0.999672,0


In [ ]:
# Connect the end points of lines that are within a threshold distance
# unique_combinations = df.drop_duplicates(subset=['r', 'g', 'b'])
# unique_combinations
# shape_starts = unique_combinations.index.tolist()
# print(shape_starts)

# The tricky bit is still the start and end of each shape - though the good news you have 
# multiple bits of information to identify them. You need to decide what to do at the end of each shape - join or jump
# First assign a shape ID to each line



# Case 1 - the start and end points of the line and its corresponding colour don't quite match up


# # Define the critical distance value
# critical_distance = 1.1

# # Find rows where distance_to_next is greater than the critical value
# greater_than_critical = df[df['distance_from_last'] > critical_distance]
# # greater_than_critical
# df.iloc[14:19]

# # Plot a histogram of the distances
# plt.hist(distances_no_nan, bins=10, edgecolor='black')
# plt.xlabel('Distance')
# plt.ylabel('Frequency')
# plt.title('Histogram of Distances Between Consecutive Points')
# plt.show()

# Display the rows that meet the condition
print(greater_than_critical)

            x         y    z    r    g    b  distance_to_next
2   -0.900000  0.000000  0.0  200    0    0          1.900000
4    0.639206 -0.316260  0.0    0  255    0          1.669435
35   0.900000  0.000000  0.0    0  255    0          1.562843
76   0.900000  0.000000  0.0    0  255    0          1.800000
102  0.284209  0.273008  0.0    0    0  255          1.215271
152  0.840359 -0.137222  0.0    0    0  255          1.528889


In [20]:
def determine_style(row):
    # capillaries
    if row['b'] > 0:
        return 2
    # venules
    elif row['g'] > 0:
        return 5
    # veins 
    elif row['r'] > 0:
        return 8 
    else:
        return 1

def thickness_to_radius(thickness):
    # Assuming radius is proportional to thickness
    return thickness / 2

# Apply determine_style to get thickness
df['thickness'] = df.apply(determine_style, axis=1)

# Convert thickness to radius3
df['radius'] = df['thickness'].apply(thickness_to_radius)

df['e'] = np.pi * df['radius']**2 * df['distance_to_next']

df

,x,y,z,r,g,b,distance_to_next,thickness,radius,e
0,0.900000,0.000000,0.0,200,0,0,NaN,8,4.0,NaN
1,1.000000,0.000000,0.0,200,0,0,0.100000,8,4.0,5.026548
2,-0.900000,0.000000,0.0,200,0,0,1.900000,8,4.0,95.504417
3,-1.000000,0.000000,0.0,200,0,0,0.100000,8,4.0,5.026548
4,0.639206,-0.316260,0.0,0,255,0,1.669435,5,2.5,32.779289
...,...,...,...,...,...,...,...,...,...,...
190,-0.087749,-0.012726,0.0,0,0,255,0.014130,2,1.0,0.044391
191,-0.100991,-0.006839,0.0,0,0,255,0.014491,2,1.0,0.045526
192,-0.114177,0.000032,0.0,0,0,255,0.014869,2,1.0,0.046712
193,-0.127244,0.007937,0.0,0,0,255,0.015272,2,1.0,0.047979


In [21]:
def generate_gcode(df):
    gcode_lines = []

    # Define some starting commands if necessary
    gcode_lines.append("G21 ; Set units to millimeters")
    gcode_lines.append("G90 ; Absolute positioning")
    
    for index, row in df.iterrows():
        # Move to the start position
        gcode_lines.append(f"G0 X{row['x']} Y{row['y']} Z{row['z']} ; Move to start position")
        
        # Example of drawing or other operation
        # For simplicity, let's assume 'e' is used for setting extrusion or other properties
        gcode_lines.append(f"G1 E{row['e']} ; Set extrusion amount")
        
        # Move to another position or perform an action (for demonstration)
        gcode_lines.append(f"G1 X{row['x']+5} Y{row['y']} Z{row['z']} ; Move to another position")
    
    # Add end commands if necessary
    gcode_lines.append("M30 ; End of program")

    return "\n".join(gcode_lines)

# Generate the G-code
gcode = generate_gcode(df)

# Save to a file
with open("output.gcode", "w") as file:
    file.write(gcode)